### Energieversorgung Vertiefung Hausübung 1: Zuverlässigkeitskennzahlen

In [112]:
import pandas as pd
from tabulate import tabulate

In [113]:
# Eingabe Matrikelnummer
# MatrNr = 11907067
# MatrNr = 12001535
MatrNr = 12143819

# Berechnung IPS
def calculation_ips(Matrikelnummer):
    q = sum(int(digit) for digit in str(Matrikelnummer))
    return ((q * 5) % 40) * 10**-4

IPS = calculation_ips(MatrNr)

# Allgemeine Funktionen
def process_values(val):
    return (0, 1) if val == "V1" else (1, 0) if val == "V2" else (1, 1)

def split_v1_v2(value):
    return int("V1" not in value), int("V2" not in value)


In [114]:
# Parameter Leitungslänge [km]
ltg_l = 5

# Parameter Ausfallhäufigkeit
h_ltg110 = 0.005 * ltg_l  # [1/a]
h_ltg20 = 0.001 * ltg_l  # [1/a]
h_T = 0.015  # [1/a]
h_TR = 0.00012 + IPS  # [1/a]
h_ls = 0.000259  # [1/a]
h_SS = 0.000148  # [1/a]

# Parameter Ausfalldauer
ta_ltg110 = 80  # [h]
ta_ltg20 = 26  # [h]
ta_T = 28  # [h]
ta_TR = 72  # [h]
ta_ls = 72  # [h]
ta_SS = 72  # [h]
ksT_UW = 0.25  # Korrekturschaltdauer im UW
ksT_aUW = 0.25  # Korrekturschaltdauer außerhalb vom UW

# Parameter Lasten [MW]
P_A = 5
P_GK = 5
P_B = 5
P_C = 5

In [ ]:
# Aktuelle Situation: Umspannwerk A in H-Schaltung, der Kunde ist mit Stichleitung (20kV 5km) an den Abgang V1 des UW A angeschlossen

# Berechnung der Situation in UW A in H-Schaltung

rows = [
    # ["LTV_20kV", "LT", "V1", "V1"], # Leitung vom Abgang V1 zum Großkunden
    ["LTG1", "LT", "V1 + V2", "-"],
    ["LTG2", "LT", "V1 + V2", "-"],
    ["HTR11", "TR", "V1 + V2", "V1"],
    ["HTR12", "TR", "V1 + V2", "V2"],
    ["HTR21", "TR", "V1 + V2", "V1"],
    ["HTR22", "TR", "V1 + V2", "V2"],
    ["QTR1", "TR", "V1 + V2", "V1"],
    ["QTR2", "TR", "V1 + V2", "V2"],
    ["HLS11", "LS", "V1 + V2", "V1"],
    ["HLS12", "LS", "V1 + V2", "V2"],
    ["T1", "T", "V1", "V1"],
    ["T2", "T", "V2", "V2"],
    ["HLS21", "LS", "V1", "V1"],
    ["HLS22", "LS", "V2", "V2"],
    ["HTR31", "TR", "V1", "V1"],
    ["HTR32", "TR", "V2", "V2"],
]

elemente_H_Schaltung = [
    # {"name": "LTG_20kV", "defizithaeufigkeit": h_ltg20 * ltg_l, "ausfalldauer": ta_ltg20}, # Leitung vom Abgang V1 zum Großkunden
    {"name": "LTG1", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "LTG2", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "HTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HLS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "HLS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HTR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

data = []
for row in rows:
    v1_1, v2_1 = split_v1_v2(row[2])
    v1_2, v2_2 = process_values(row[3])
    entry = next(item for item in elemente_H_Schaltung if item["name"] == row[0])
    data.append([row[0], row[1], v1_1, v2_1, v1_2, v2_2, entry["defizithaeufigkeit"], entry["ausfalldauer"]])

columns = ["Name", "Type", "V1_1", "V2_1", "V1_2", "V2_2", "Defizithäufigkeit (H in 1/a)", "Ausfalldauer (in h)"]

pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)       # Erhöht die Breite für die Ausgabe
df = pd.DataFrame(data, columns=columns)

# Verbraucher 1: Wenn dieser nach der Korrektuschaltung wiederversorgt ist, muss man für NV, P und E nur die 0,25h Korrekturzeit berücksichtigen
# ansonsten muss die Ausfalldauer UND die 0,25h berücksichtigt werden
# Initialisiere die neuen Spalten mit NaN (falls sie noch nicht existieren)
df['NV_V1'] = None
df['Defizitleistung P_V1'] = None
df['Defizitenergie E_V1'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V1_2"] == 0:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V1_2"] == 1:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * 0.25
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * 0.25

# Selbe Herangehensweise wie bei Verbraucher 2
df['NV_V2'] = None
df['Defizitleistung P_V2'] = None
df['Defizitenergie E_V2'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V2_2"] == 0:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V2_2"] == 1:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * 0.25
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * 0.25

# Summierung der Werte der jeweiligen Spalten für die H-Schaltung
Nichtverlaesslichkeit_V1 = round(df["NV_V1"].sum(), 4)
Nichtverlaesslichkeit_V2 = round(df["NV_V2"].sum(), 4)
Defizitleistung_V1 = round(df["Defizitleistung P_V1"].sum(), 4)
Defizitleistung_V2 = round(df["Defizitleistung P_V2"].sum(), 4)
Defizitenergie_V1 = round(df["Defizitenergie E_V1"].sum(), 4)
Defizitenergie_V2 = round(df["Defizitenergie E_V2"].sum(), 4) 

# Exportiere als csv-Datei
df.iloc[:, :6].to_csv('output_H_Schaltung.csv', sep=';', index=False)
print(df)
print(" ")

# Ausgeben der summierten Werte
print(f"Nichtverlässlichkeit V1: {Nichtverlaesslichkeit_V1} h/a")
print(f"Nichtverlässlichkeit V2: {Nichtverlaesslichkeit_V2} h/a")
print(f"Defizitleistung V1: {Defizitleistung_V1} MW")
print(f"Defizitleistung V2: {Defizitleistung_V2} MW")
print(f"Defizitenergie V1: {Defizitenergie_V1} MWh")
print(f"Defizitenergie V2: {Defizitenergie_V2} MWh")
print(" ")
print(f"Nichtverlässlichkeit Gesamt: {Nichtverlaesslichkeit_V1 + Nichtverlaesslichkeit_V2} 1/a")
print(f"Defizitleistung Gesamt: {Defizitleistung_V1 + Defizitleistung_V2} MW")
print(f"Defizitenergie Gesamt: {Defizitenergie_V1 + Defizitenergie_V2} MWh")

     Name Type  V1_1  V2_1  V1_2  V2_2  Defizithäufigkeit (H in 1/a)  Ausfalldauer (in h)     NV_V1 Defizitleistung P_V1 Defizitenergie E_V1     NV_V2 Defizitleistung P_V2 Defizitenergie E_V2
0    LTG1   LT     0     0     1     1                      0.125000                   80   0.03125                0.625             0.15625   0.03125                0.625             0.15625
1    LTG2   LT     0     0     1     1                      0.125000                   80   0.03125                0.625             0.15625   0.03125                0.625             0.15625
2   HTR11   TR     0     0     0     1                      0.002620                   72  0.189295               0.0131            0.946475  0.000655               0.0131            0.003275
3   HTR12   TR     0     0     1     0                      0.002620                   72  0.000655               0.0131            0.003275  0.189295               0.0131            0.946475
4   HTR21   TR     0     0     0     1  

In [116]:
# Berechnung für Doppelsammelschienen-Topologie

elemente_DSS_Schaltung = [
    {"name": "LTG1", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "LTG2", "defizithaeufigkeit": h_ltg110 * ltg_l, "ausfalldauer": ta_ltg110},
    {"name": "LTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "STR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QLS", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "SS1", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "SS2", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "STR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR41", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR42", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "LS31", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS32", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "ATR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "ATR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

# Beschreibung der Ausfallwirkung für die Doppelsammelschienen-Schaltanlage in einer Tabelle
rows = []

# Der in der Abbildung ersichtliche Schaltzustand wird als Basis für nachfolgende Überlegungen getroffen

# Zeilenweise Eintragen der Werte
rows.append(["LTG1", "LT", "-", "-"])
rows.append(["LTG2", "LT", "-", "-"])
rows.append(["LTR11", "TR", "-", "-"])
rows.append(["LTR12", "TR", "-", "-"])
rows.append(["LS11", "LS", "V1 + V2", "-"])
rows.append(["LS12", "LS", "V1 + V2", "-"])

rows.append(["STR11", "TR", "V1 + V2", "-"])
rows.append(["STR12", "TR", "V1 + V2", "-"]) 

rows.append(["STR21", "TR", "V1 + V2", "-"]) 
rows.append(["STR22", "TR", "V1 + V2", "-"]) 

rows.append(["QTR1", "TR", "V1 + V2", "-"])
rows.append(["QTR2", "TR", "V1 + V2", "-"])
rows.append(["QLS", "LS", "V1 + V2", "-"])

rows.append(["SS1", "SS", "V1 + V2", "-"])
rows.append(["SS2", "SS", "V1 + V2", "-"])

rows.append(["STR31", "TR", "V1 + V2", "V1"])
rows.append(["STR32", "TR", "V1 + V2", "V2"])

rows.append(["STR41", "TR", "V1 + V2", "V1"])
rows.append(["STR42", "TR", "V1 + V2", "V2"])

rows.append(["LS21", "LS", "V1 + V2", "V1"])
rows.append(["LS22", "LS", "V1 + V2", "V2"])

rows.append(["T1", "T", "V1", "V1"])
rows.append(["T2", "T", "V2", "V2"])

rows.append(["LS31", "LS", "V1", "V1"])
rows.append(["LS32", "LS", "V2", "V2"])

rows.append(["ATR11", "TR", "V1", "V1"])
rows.append(["ATR12", "TR", "V2", "V2"])

# Selbes Procedere wie bei H-Schaltung
data = []
for row in rows:
    v1_1, v2_1 = split_v1_v2(row[2])
    v1_2, v2_2 = process_values(row[3])
    entry = next(item for item in elemente_DSS_Schaltung if item["name"] == row[0])
    data.append([row[0], row[1], v1_1, v2_1, v1_2, v2_2, entry["defizithaeufigkeit"], entry["ausfalldauer"]])

columns = ["Name", "Type", "V1_1", "V2_1", "V1_2", "V2_2", "Defizithäufigkeit (H in 1/a)", "Ausfalldauer (in h)"]

pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)       # Erhöht die Breite für die Ausgabe
df = pd.DataFrame(data, columns=columns)

# Verbraucher 1: Wenn dieser nach der Korrektuschaltung wiederversorgt ist, muss man für NV, P und E nur die 0,25h Korrekturzeit berücksichtigen
# ansonsten muss die Ausfalldauer UND die 0,25h berücksichtigt werden
# Initialisiere die neuen Spalten mit NaN (falls sie noch nicht existieren)
df['NV_V1'] = None
df['Defizitleistung P_V1'] = None
df['Defizitenergie E_V1'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V1_2"] == 0:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V1_2"] == 1:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * 0.25
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * 0.25


# Selbe Herangehensweise wie bei Verbraucher 2
df['NV_V2'] = None
df['Defizitleistung P_V2'] = None
df['Defizitenergie E_V2'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V2_2"] == 0:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V2_2"] == 1:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * 0.25
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * P_A
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * 0.25

# Summierung der Werte der jeweiligen Spalten für die H-Schaltung
Nichtverlaesslichkeit_V1 = round(df["NV_V1"].sum(), 4)
Nichtverlaesslichkeit_V2 = round(df["NV_V2"].sum(), 4)
Defizitleistung_V1 = round(df["Defizitleistung P_V1"].sum(), 4)
Defizitleistung_V2 = round(df["Defizitleistung P_V2"].sum(), 4)
Defizitenergie_V1 = round(df["Defizitenergie E_V1"].sum(), 4)
Defizitenergie_V2 = round(df["Defizitenergie E_V2"].sum(), 4)

# Exportiere als csv-Datei
df.iloc[:, :6].to_csv('output_DSS_Schaltung.csv', sep=';', index=False)
print(df)
print(" ")

# Ausgeben der summierten Werte
print(f"Nichtverlässlichkeit V1: {Nichtverlaesslichkeit_V1} 1/a")
print(f"Nichtverlässlichkeit V2: {Nichtverlaesslichkeit_V2} 1/a")
print(f"Defizitleistung V1: {Defizitleistung_V1} MW")
print(f"Defizitleistung V2: {Defizitleistung_V2} MW")
print(f"Defizitenergie V1: {Defizitenergie_V1} MWh")
print(f"Defizitenergie V2: {Defizitenergie_V2} MWh")
print(" ")
print(f"Nichtverlässlichkeit Gesamt: {Nichtverlaesslichkeit_V1 + Nichtverlaesslichkeit_V2} 1/a")
print(f"Defizitleistung Gesamt: {Defizitleistung_V1 + Defizitleistung_V2} MW")
print(f"Defizitenergie Gesamt: {Defizitenergie_V1 + Defizitenergie_V2} MWh")

     Name Type  V1_1  V2_1  V1_2  V2_2  Defizithäufigkeit (H in 1/a)  Ausfalldauer (in h)     NV_V1 Defizitleistung P_V1 Defizitenergie E_V1     NV_V2 Defizitleistung P_V2 Defizitenergie E_V2
0    LTG1   LT     1     1     1     1                      0.125000                   80   0.03125                0.625             0.15625   0.03125                0.625             0.15625
1    LTG2   LT     1     1     1     1                      0.125000                   80   0.03125                0.625             0.15625   0.03125                0.625             0.15625
2   LTR11   TR     1     1     1     1                      0.002620                   72  0.000655               0.0131            0.003275  0.000655               0.0131            0.003275
3   LTR12   TR     1     1     1     1                      0.002620                   72  0.000655               0.0131            0.003275  0.000655               0.0131            0.003275
4    LS11   LS     0     0     1     1  